In [25]:
#Goal Tracker - 1 goal for MVP
import datetime 

goal = "Self-care: Get a hair cut"
deadline_str = "06/04/2025"
deadline_datetime = datetime.datetime.strptime(deadline_str, "%d/%m/%Y").date()

print(deadline_datetime)
print(goal)

#Checking current date
current_date = datetime.date.today()
print(current_date)

#Checking deadline against current date
if deadline_datetime < current_date:
    print(f"Your goal:  {goal} is due since {deadline_datetime}. You are better than this.")
elif deadline_datetime == current_date:
    print(f"Your goal:  {goal} is due today {deadline_datetime}. Go get it.")
else:
    print(f"Your goal: {goal} is due on {deadline_datetime}. Are you going to get it?.")



2025-04-06
Self-care: Get a hair cut
2025-04-05
Your goal: Self-care: Get a hair cut is due on 2025-04-06. Are you going to get it?.


In [ ]:
#Let user input their goals and deadline

import datetime 

goals = []

while True:
    goal_description = input("Enter your goal description (or 'done' to finish): ")
    if goal_description == "done":
        break
    

    while True:
        goal_deadline_str = input("Enter a deadline for your {goal_description} Use DD-MM-YY: ")
        try:
            goal_deadline_datetime = datetime.datetime.strptime(goal_deadline_str, "%d/%m/%Y").date()
            goals.append({"description": goal_description, "deadline": goal_deadline_datetime})
            print("Goals all set")
            break #breaks out of the inner loop
        except ValueError:
            print("Invalid date format: Please use DD/MM/YYYY format")

current_date = datetime.date.today()

print("\n--- Goal Tracking ---")

if not goals:
    print("No goals entered.")
else:
    for goal in goals:
        goal_description = goal["description"]
        goal_deadline = goal["deadline"]
        print(goal_description)
        print(goal_deadline)

    #Checking deadline against current date
    if goal_deadline < current_date:
        print(f"Your goal:  {goal_description} is due since {goal_deadline}. You are better than this.")
    elif goal_deadline == current_date:
        print(f"Your goal:  {goal_description} is due today {goal_deadline}. Go get it.")
    else:
        print(f"Your goal: {goal_description} is due on {goal_deadline}. Are you going to get it?.")



In [ ]:
#Create App.py using Flask for a Web Interface
import sqlite3
import datetime
from flask import Flask, render_template, request, redirect, url_for, g # Import 'g'

app = Flask(__name__)

"""
goals = [] #Use a database later

#Creating a home page for all the goals and its status
@app.route('/')
def index():
    current_date = datetime.date.today()
    for goal in goals:
        goal['status'] = "Upcoming"
        if goal['deadline'] < current_date:
            goal['status'] = "Past Due"
        elif goal['deadline'] == current_date:
            goal['status'] = "Due Today"
    return render_template('index.html', goals=goals)
"""

#Implementing a SQLITE3 Database to store goals

DATABASE = 'goals.db'

def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect(DATABASE)
        g.db.row_factory = sqlite3.Row # Allows accessing columns by name
    return g.db

def close_db():
    db = g.pop('db', None)
    if db is not None:
        db.close()

def init_db():
    with app.app_context():
        db = get_db()
        with app.open_resource('schema.sql', mode='r') as f:
            db.cursor().executescript(f.read())
        db.commit()

@app.cli.command('initdb')
def initdb_command():
    """Initializes the database."""
    init_db()
    print('Initialized the database.')

@app.teardown_appcontext
def teardown_db(error):
    close_db()



#Creating a home page for all the goals and its status
"""
@app.route('/')
def index():
    current_date = datetime.date.today()
    for goal in goals:
        goal['status'] = "Upcoming"
        if goal['deadline'] < current_date:
            goal['status'] = "Past Due"
        elif goal['deadline'] == current_date:
            goal['status'] = "Due Today"
    return render_template('index.html', goals=goals)
"""
@app.route('/add', methods=["GET", "POST"])
def add_goal():
    if request.method == "POST":
        description = request.form['description']
        deadline_str = request.form['deadline']
        db = get_db()
        try:
            deadline = datetime.datetime.strptime(deadline_str, "%Y-%m-%d").date() # Assuming your HTML form uses YYYY-MM-DD
            db.execute('INSERT INTO goals (description, deadline) VALUES (?, ?)',
                       (description, deadline.strftime('%Y-%m-%d')))
            db.commit()
            return redirect(url_for('index'))
        except ValueError:
            return "Invalid date format"
        finally:
            close_db() # Explicitly close the connection for now, though teardown will also handle it
    return render_template('add.html')

"""
@app.route('/add', methods=["GET", "POST"])
def add_goal():
    if request.method == "POST":
        description = request.form['description']
        deadline_str = request.form['deadline']

        try:
            deadline = datetime.datetime.strptime(deadline_str, "%Y-%m-%d").date() # Adjust format as needed
            goals.append({'description': description, 'deadline': deadline})
            return redirect(url_for('index'))
        except ValueError:
            return "Invalid date format" # Better error handling needed

    return render_template('add.html')
"""
@app.route('/')
def index():
    conn = get_db()
    goals_db = conn.execute('SELECT id, description, deadline FROM goals').fetchall()
    conn.close()
    goals = []
    current_date = datetime.date.today()
    for row in goals_db:
        deadline_str = row['deadline']
        deadline = datetime.datetime.strptime(deadline_str, '%Y-%m-%d').date()
        status = "Upcoming"
        if deadline < current_date:
            status = "Past Due"
        elif deadline == current_date:
            status = "Due Today"
        goals.append({'id': row['id'], 'description': row['description'], 'deadline': deadline, 'status': status})
    return render_template('index.html', goals=goals)

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


In [2]:
import sqlite3
import pandas as pd

# Specify the path to your database file
db_file = 'goals.db'

try:
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Execute the SQL query to select all data from the 'goals' table
    cursor.execute("SELECT * FROM goals")

    # Fetch all the results
    results = cursor.fetchall()

    # Get the column names from the cursor description
    column_names = [description[0] for description in cursor.description]

    # If you have pandas installed, you can display the results in a DataFrame
    if 'pandas' in globals():
        df = pd.DataFrame(results, columns=column_names)
        print(df)
    else:
        # If pandas is not installed, print the results as a list of tuples
        for row in results:
            print(row)

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    # Close the database connection
    if conn:
        conn.close()

(5, 'Get a haircut', '2025-04-10')
(6, 'Jump off a plane', '2025-04-25')
